### Import packages and setup

In [2]:
# for package auto reload
%load_ext autoreload
%autoreload 2

# for better rendering of plots in jupyter notebook
%matplotlib inline

In [3]:
# base modules
import os
import sys
import copy

# for manipulating data
import numpy as np
import pandas as pd
import math
import dill
import gc

# for Machine Learning
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, BaggingRegressor
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree

# for visualization
from IPython.display import display
from matplotlib import pyplot as plt

In [4]:
# path to repo
path_to_repo = os.path.dirname(os.getcwd())
path_to_repo

'/Users/sonyakurakova/Desktop/em/Y2/MLP-2024'

### Load and preprocess data

In [5]:
types = {#'id': 'int64', 
         #'int_nbr': 'int32',
         #'store_nbr': 'int8',
         #'unit_sales': 'float32',
         #'onpromotion': 'object'} # boolean with missing values

SyntaxError: incomplete input (3134252354.py, line 5)

In [6]:
%%time
df_all = pd.read_csv('Motor_Vehicle_Collisions_-_Crashes_20241014.csv', #../ helps to go down the level to find the dataset
                     parse_dates=['CRASH DATE'], 
                     #dtype=types, 
                     infer_datetime_format=True)
df_all

<timed exec>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 13.2 s, sys: 7.08 s, total: 20.2 s
Wall time: 32.5 s


,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2021-09-11,2:39,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,2022-03-26,11:45,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,2022-06-29,6:55,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,2021-09-11,9:35,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,2021-12-14,8:13,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2125758,2024-08-20,15:18,BROOKLYN,11212.0,NaN,NaN,NaN,NaN,NaN,44 NEW LOTS AVENUE,...,Unspecified,NaN,NaN,NaN,4750415,Sedan,Box Truck,NaN,NaN,NaN
2125759,2024-08-21,8:50,BRONX,10475.0,NaN,NaN,NaN,NaN,NaN,2035 BARTOW AVENUE,...,Unspecified,NaN,NaN,NaN,4750338,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
2125760,2024-08-21,20:10,QUEENS,11103.0,NaN,NaN,NaN,BROADWAY,NEWTOWN ROAD,NaN,...,Unspecified,NaN,NaN,NaN,4750194,Station Wagon/Sport Utility Vehicle,Moped,NaN,NaN,NaN
2125761,2024-08-21,12:58,BRONX,10462.0,NaN,NaN,NaN,WHITE PLAINS ROAD,WOOD AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4750059,Bike,Sedan,NaN,NaN,NaN


In [8]:
df_all["LATITUDE"].value_counts()

0.000000     4786
40.861862     900
40.696033     779
40.804700     692
40.608757     671
             ... 
40.797680       1
40.601913       1
40.575733       1
40.683543       1
40.804930       1
Name: LATITUDE, Length: 127018, dtype: int64

In [9]:
print(df_all.isnull().sum())

CRASH DATE                             0
CRASH TIME                             0
BOROUGH                           661104
ZIP CODE                          661364
LATITUDE                          252597
LONGITUDE                         252597
LOCATION                          252597
ON STREET NAME                    455112
CROSS STREET NAME                 810044
OFF STREET NAME                  1763024
NUMBER OF PERSONS INJURED             18
NUMBER OF PERSONS KILLED              31
NUMBER OF PEDESTRIANS INJURED          0
NUMBER OF PEDESTRIANS KILLED           0
NUMBER OF CYCLIST INJURED              0
NUMBER OF CYCLIST KILLED               0
NUMBER OF MOTORIST INJURED             0
NUMBER OF MOTORIST KILLED              0
CONTRIBUTING FACTOR VEHICLE 1       7144
CONTRIBUTING FACTOR VEHICLE 2     333073
CONTRIBUTING FACTOR VEHICLE 3    1972957
CONTRIBUTING FACTOR VEHICLE 4    2091067
CONTRIBUTING FACTOR VEHICLE 5    2116325
COLLISION_ID                           0
VEHICLE TYPE COD

In [11]:
for i in df_all.columns:
    null_rate = df_all[i].isnull().sum()/len(df_all) *100
    if null_rate > 0:
        print(f"{i} null rate: {null_rate:.2f}%")

BOROUGH null rate: 31.10%
ZIP CODE null rate: 31.11%
LATITUDE null rate: 11.88%
LONGITUDE null rate: 11.88%
LOCATION null rate: 11.88%
ON STREET NAME null rate: 21.41%
CROSS STREET NAME null rate: 38.11%
OFF STREET NAME null rate: 82.94%
NUMBER OF PERSONS INJURED null rate: 0.00%
NUMBER OF PERSONS KILLED null rate: 0.00%
CONTRIBUTING FACTOR VEHICLE 1 null rate: 0.34%
CONTRIBUTING FACTOR VEHICLE 2 null rate: 15.67%
CONTRIBUTING FACTOR VEHICLE 3 null rate: 92.81%
CONTRIBUTING FACTOR VEHICLE 4 null rate: 98.37%
CONTRIBUTING FACTOR VEHICLE 5 null rate: 99.56%
VEHICLE TYPE CODE 1 null rate: 0.68%
VEHICLE TYPE CODE 2 null rate: 19.43%
VEHICLE TYPE CODE 3 null rate: 93.08%
VEHICLE TYPE CODE 4 null rate: 98.43%
VEHICLE TYPE CODE 5 null rate: 99.57%


- drop columns with a very high null amount: vehicle type code 5,4?
- do I need to drop the vehicles 3,4,5 from the model? 
- maybe to clear the data: if the "on street is empty", replace it by the "off street"
- model to do: 
    - predict the severity of collision 
    - predict if a collision will result in fatalities or injures (people injured, killed)
    - predict when and where the accidents are likely to occur (date, time, location)
    - predict the likely contributing factors for a collision 
    - predict the number of vehicles involved in the crash based on location, time and contributing factors
    - identify and predict hotspots where collisions are likely to happen in the future 
    - predict high-risk behaviors in drivers based on patterns in historical collision data
- geographical information analysis?
- natural text analysis? 

In [13]:
df_all["CONTRIBUTING FACTOR VEHICLE 1"].unique()

array(['Aggressive Driving/Road Rage', 'Pavement Slippery',
       'Following Too Closely', 'Unspecified', nan, 'Passing Too Closely',
       'Driver Inexperience', 'Passing or Lane Usage Improper',
       'Turning Improperly', 'Unsafe Lane Changing', 'Unsafe Speed',
       'Reaction to Uninvolved Vehicle', 'Steering Failure',
       'Traffic Control Disregarded', 'Other Vehicular',
       'Driver Inattention/Distraction', 'Oversized Vehicle',
       'Pedestrian/Bicyclist/Other Pedestrian Error/Confusion',
       'Alcohol Involvement', 'View Obstructed/Limited',
       'Failure to Yield Right-of-Way', 'Illnes', 'Lost Consciousness',
       'Brakes Defective', 'Backing Unsafely', 'Glare',
       'Passenger Distraction', 'Fell Asleep', 'Obstruction/Debris',
       'Tinted Windows', 'Animals Action', 'Drugs (illegal)',
       'Pavement Defective', 'Other Lighting Defects',
       'Outside Car Distraction', 'Driverless/Runaway Vehicle',
       'Tire Failure/Inadequate', 'Fatigued/Drowsy',


In [15]:
df_all.columns

Index(['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE',
       'LONGITUDE', 'LOCATION', 'ON STREET NAME', 'CROSS STREET NAME',
       'OFF STREET NAME', 'NUMBER OF PERSONS INJURED',
       'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED',
       'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED',
       'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED',
       'NUMBER OF MOTORIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1',
       'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3',
       'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5',
       'COLLISION_ID', 'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2',
       'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5'],
      dtype='object')

In [16]:
df_all["LOCATION"].value_counts()

(0.0, 0.0)                 4786
(40.861862, -73.91282)      685
(40.608757, -74.038086)     670
(40.696033, -73.98453)      646
(40.8047, -73.91243)        597
                           ... 
(40.854046, -73.9303)         1
(40.743027, -73.86509)        1
(40.63238, -74.07576)         1
(40.593765, -73.79426)        1
(40.82961, -73.88756)         1
Name: LOCATION, Length: 287970, dtype: int64

In [17]:
df_all["BOROUGH"].value_counts()

BROOKLYN         466932
QUEENS           392870
MANHATTAN        326661
BRONX            216787
STATEN ISLAND     61409
Name: BOROUGH, dtype: int64

based on the location and the day of the week predict the number of crashes in a day. 